In [1]:
import numpy as np
import scipy as sp
from scipy import fftpack

import pandas as pd

import os
import glob
from sklearn.preprocessing import StandardScaler


In [2]:
path = os.getcwd() + '\\data\\'
print(os.getcwd())
extension = 'csv'

os.chdir(path)
titles = glob.glob('*.{}'.format(extension))

C:\Users\tlpel\Documents\Studie\AI1\pAI\code


In [3]:
datasets = []
    
for d in titles:
    data = pd.read_csv(path + d)
    if len(data.columns) == 1:
        data = pd.read_csv(path + d, sep=";")
    data['action'] = data['Stimulus'].apply(lambda x: x.replace(' ', '.').split('.')[0])
    datasets.append(data[data.columns[1:]])

In [15]:
from sklearn.decomposition import PCA
X = datasets[0].ix[:,1:15].values
print(np.array(X).shape)
pca = PCA(n_components=14)
Z = pca.fit_transform(np.array(X))
for i in range(Z.shape[1]):
    datasets[0].ix[:,i+1] = Z[:,i]
print(datasets[0])


(11288, 14)
                    Stimulus         AF3          F7         F3         FC5  \
0      crouch - Copy (2).png -558.339302  -81.255286 -56.637044 -106.184943   
1      crouch - Copy (2).png -534.274842  -89.382375 -41.621680  -98.718844   
2      crouch - Copy (2).png -553.094837  -90.090383 -52.386219  -91.233122   
3      crouch - Copy (2).png -563.766718  -92.338583 -54.576909  -91.416527   
4      crouch - Copy (2).png -566.214397  -97.081760 -43.778743  -91.963732   
5      crouch - Copy (2).png -585.132712  -95.823024 -46.196693  -86.514537   
6      crouch - Copy (2).png -587.951343  -89.744517 -46.953983  -81.158017   
7      crouch - Copy (2).png -567.787206  -85.223171 -35.768550  -80.503574   
8      crouch - Copy (2).png -562.425427  -83.392079 -31.711467  -80.740985   
9      crouch - Copy (2).png -570.912937  -83.568891 -33.794455  -79.978500   
10     crouch - Copy (2).png -558.260836  -82.253762 -31.790966  -80.886126   
11     crouch - Copy (2).png -537.427763

In [5]:
def extract_action(stimulus):
    action_nr = stimulus.split('.')[0]
    splitted = action_nr.split('-')
    action = splitted[0]
    if len(splitted) == 1:
        nr = '0'
    else:
        nr = splitted[1]
    return action.strip(), nr.strip()

In [6]:
actions = {}

data = datasets[0]
data = data[data.columns[1:]]

data = data['action']
i = 0

for d in data:
    action,number = extract_action(d)

    if not action in list(actions.keys()):
        actions[action] = i
        i += 1
        print('added', action)
        
action_length = len(actions)

added crouch
added walk
added run
added jump


In [7]:
# Xs = []
# X = None
# Ys = None
# lowest = float('inf')

# for i in range(len(datasets)):
#     data = datasets[i]
#     rows = data.iterrows() 
#     j = 0
#     previous_action = None
#     previous_number = None
#     for i in range(data.shape[0]):
#         row = next(rows)[1]
#         x = np.array(row[2:])
#         action,number = extract_action(row[1])
#         y = actions[action]

#         if previous_action == None:
#             previous_action = action

#         if previous_number == None:
#             previous_number = number


#         if not action == previous_action or not number == previous_number:

#             if len(X) < lowest:
#                 lowest = len(X)

#             X = None

#         if X == None:
#             X = x
#         else:
#             X = np.append(X,x) 

#         previous_action = action
#         previous_number = number

# print('Shortest signal length is:', lowest)

In [8]:
# Xs = None
# X = None
# Ys = None

# for i in range(len(datasets)):
#     data = datasets[i]
#     rows = data.iterrows() 
#     j = 0
#     previous_action = None
#     previous_number = None
#     for i in range(data.shape[0]):
#         row = next(rows)[1]
#         x = np.array(row[2:])
#         action,number = extract_action(row[1])
#         y = actions[action]

#         if previous_action == None:
#             previous_action = action

#         if previous_number == None:
#             previous_number = number


#         if not action == previous_action or not number == previous_number:

#             X = X[:lowest]

#             if Xs == None:
#                 Xs = X
#             else:
#                 Xs = np.vstack((Xs,X))

#             if Ys == None:
#                 Ys = np.array(y)
#             else:
#                 Ys = np.append(Ys,y)

#             X = None

#         if X == None:
#             X = x
#         else:
#             X = np.append(X,x) 

#         previous_action = action
#         previous_number = number
    
    
# #     cutoff start
# #     if not previous_action == None and not strcmp(action,previous_action):
# #         if j > 40:
# #             while j < 70:
# #                 x = np.zeros()
# #                 X.append(x)
# #             Xs.append(X)
# #             Ys.append(y)
# #             X = None
# #             previous_action = None
# #         continue
            

# #     index = actions[action]
# #     y[index] = 1
    
# #     if X == None:
# #         X = x
# #     else:
# #         X.append(x)
    
# #     previous_action = action
    
# #     if j == 70:
# #         j = 0
# #         Xs.append(X)
# #         Ys.append(y)
# #         previous_action = None
# #         X = None
        
# # if j > 40:
# #     while j < 70:
# #         x = np.zeros()
# #         X.append(x)
# #     Xs.append(X)
# #     Ys.append(y)
    
# print(Xs.shape)
# print(Ys.shape)



In [9]:
## Split the signals into small subsets with an overlap
def subset_data(data, ss_dim = None, ss_num = 10, overlap = .5, cut_smaller = True):
    if overlap > 1: # in case the value passed is a percentage
        overlap = float(overlap)/100
        
    if ss_dim is None: # either choose the dimension of the subsets or the number
        ss_dim = int(len(data)/(ss_num*overlap)) # by default it will divide the signal in 10 subsets
    
    subsets = []
    i = len(data) - 1
    while i >= 0:
        j = max(i - ss_dim, 0)
        subsets.append(data[j:i])
        i -= int(ss_dim * (1 - overlap))
    
    while cut_smaller and len(subsets[-1]) < ss_dim :
        subsets = subsets[:-1]
        
    return np.array(subsets)

def prepare_data(dataframe, ss_dim = 20, ss_num = 168, overlap = .5):
    dataset = {}
    for action in set(dataframe.action.values):
        a = []
        data = dataframe[dataframe.action == action].copy()
        for c in data.columns[1:-1]:
            a.append(np.array(subset_data(np.array(data[c]), ss_dim, ss_num, overlap, cut_smaller = True)))
        dataset[action] = [np.asmatrix(d).flatten() for d in np.transpose(np.array(a), [1, 2, 0])] # if you want a list of matrixes
        #dataset[action] = np.transpose(np.array(a), [1, 2, 0]) # if you want a tensor
    return dataset

In [10]:
for data in datasets:
    dataset = prepare_data(data, ss_dim = 1, overlap = 0)
    break

In [11]:
X = None
Y = None

for t in dataset.keys():
    
    x = np.squeeze(dataset[t], axis = 1)
    
    if X is None:
        X = x   
    else:
        X = np.concatenate( (X, x))
        
        
    y = np.array([actions[t]])
    y = np.tile(y,len(x))
    
    if Y is None:
        Y = y
    else:
        Y = np.concatenate((Y,y))

X = StandardScaler().fit_transform(X)
print(len(X))
print(len(Y))

11284
11284


In [12]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from random import shuffle

#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

cutoff = round(len(X) * 0.)  #80-20 division
Z = list(zip(X,Y))
shuffle(Z)

X,Y = zip(*Z)
print(Y)


X_train = X[0:cutoff]
y_train = Y[:cutoff]
X_test = X[cutoff: -1]
y_test = Y[cutoff: -1]


inp = X_train
out = y_train

#print(out)

def classify_net(inp, out):
    clf = MLPClassifier(solver="lbfgs", alpha=1e-4,hidden_layer_sizes=(100),random_state=1, max_iter=2000)
    print(clf.fit(inp, out))
    #print(clf.predict(inp))
    print(clf.score(X_test,y_test))
    
def classify_knn(inp, out, n):
    clf = KNeighborsClassifier(n_neighbors=n, weights='distance')
    print(clf.fit(inp, out))
    #print(clf.predict(inp))
    print(clf.score(X_test,y_test))
    
    
classify_net(inp,out)
for i in range(1,10):
    print(i)
    classify_knn(inp,out,i)

(3, 1, 0, 2, 0, 3, 1, 0, 0, 0, 2, 3, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 0, 3, 1, 0, 0, 1, 1, 1, 0, 3, 0, 1, 3, 1, 1, 3, 2, 1, 1, 0, 3, 1, 2, 1, 1, 0, 3, 2, 3, 2, 3, 3, 1, 2, 2, 3, 1, 2, 1, 3, 2, 3, 0, 2, 3, 1, 0, 2, 3, 2, 0, 3, 2, 1, 1, 0, 3, 2, 0, 1, 0, 0, 2, 1, 3, 1, 1, 3, 2, 0, 3, 1, 0, 0, 1, 2, 2, 2, 1, 2, 3, 3, 1, 1, 2, 3, 3, 3, 1, 2, 1, 2, 3, 1, 2, 2, 2, 1, 3, 0, 0, 1, 2, 0, 2, 0, 2, 0, 3, 2, 0, 3, 2, 0, 2, 0, 3, 0, 3, 3, 0, 2, 0, 1, 3, 0, 1, 1, 3, 2, 0, 0, 1, 2, 1, 0, 2, 3, 3, 3, 0, 2, 2, 3, 2, 3, 1, 3, 3, 0, 0, 2, 2, 2, 3, 2, 1, 3, 1, 0, 2, 1, 1, 2, 2, 2, 2, 0, 3, 1, 1, 2, 1, 0, 0, 1, 0, 2, 0, 2, 0, 3, 2, 3, 0, 3, 0, 1, 2, 1, 2, 1, 3, 2, 2, 1, 3, 3, 3, 3, 1, 3, 0, 1, 3, 3, 2, 1, 1, 3, 3, 1, 0, 0, 2, 2, 3, 2, 3, 3, 3, 0, 0, 2, 2, 0, 0, 0, 1, 3, 0, 2, 2, 3, 0, 2, 2, 1, 3, 3, 0, 3, 2, 3, 0, 2, 0, 3, 2, 0, 3, 3, 1, 1, 1, 3, 2, 1, 1, 0, 3, 0, 2, 3, 1, 3, 0, 1, 2, 3, 3, 1, 1, 3, 3, 1, 3, 3, 1, 1, 3, 0, 3, 2, 0, 1, 1, 2, 2, 1, 3, 0, 2, 1, 1, 2, 1, 3, 0, 3, 3, 2, 1, 3, 1, 2, 3, 2, 1, 0, 

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


ValueError: Found array with 0 feature(s) (shape=(1, 0)) while a minimum of 1 is required.